# Building a applied research lab with Group Chat

In [ ]:
## import required libraries
import os
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager
from autogen import ConversableAgent
from dotenv import load_dotenv

load_dotenv(override=True) # load environment variables from .env file

In [ ]:
## llm configuration
llm_config ={
                "model": "gpt-4o",
                "api_key": os.getenv("OPENAI_API_KEY")
            }

## The task!

In [ ]:
task = """Write a detailed weather analysis report for New York City
          over the past week. Today's date is 2024-08-23Let's 

## Create a group chat

This group chat will include these agents:

- User_proxy or Admin: to allow the user to comment on the report and ask the writer to refine it.
- Planner: to determine relevant information needed to complete the task.
- Engineer: to write code using the defined plan by the planner.
- Executor: to execute the code written by the engineer.
- Writer: to write the report.

In [ ]:
## user or admin agent
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
    lm_config=llm_config,
    human_input_mode="ALWAYS",
)

In [ ]:
planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
            The plan may involve an engineer who can write code and a scientist who doesn't write code.
            Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
            """,
    llm_config=llm_config
)

In [ ]:
## engineer agent
engineer = AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. 
    Wrap the code in a code block that specifies the script type. The user can't modify your code. 
    So do not suggest incomplete code which requires others to modify. 
    Don't use a code block if it's not intended to be executed by the executor.
    Don't include multiple code blocks in one response. 
    Do not ask others to copy and paste the result. 
    Check the execution result returned by the executor.
    If the result indicates there is an error, fix the error and output the code again. 
    Suggest the full code instead of partial code or code changes. 
    If the error can't be fixed or if the task is not solved even after the code is executed successfully, 
    analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
    """,
)

In [ ]:
## executor agent
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },
)

In [ ]:
## scientist agent
scientist_writer = autogen.AssistantAgent(
    name="Scientist/Writer",
    llm_config=gpt4_config,
    system_message="""Scientist/Writer. 
    You follow an approved plan. 
    Your role is to analyze and summarize research papers collected by the engineer. 
    You provide scientific expertise, interpret results, and write comprehensive summaries and analyses. 
    You can also suggest areas for further research or data collection.""",
)